In [2]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tqdm.notebook import tqdm
import pickle


In [3]:
vgg_model = VGG16() 
# restructure the model
vgg_model = Model(inputs=vgg_model.inputs,outputs=vgg_model.layers[-2].output)
#summarize
print(vgg_model.summary())



I0000 00:00:1735099094.725528    1175 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 146s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
features={}
try:
    with open('features.pkl', 'rb') as f:
        features = pickle.load(f)
        print("Features loaded successfully.")
except EOFError:
    print("The file is empty or corrupted. Please regenerate the pickle file.")

with open('captions.txt','r') as f:
    next(f)
    captions_doc=f.read()
    print("Captions loaded successfully")

Features loaded successfully.
Captions loaded successfully


In [5]:
mapping={}
#process lines
for line in tqdm(captions_doc.split('\n')):
    #split the line by comma
    tokens=line.split(',')
    if len(line)<2:
        continue
    image_id,caption=tokens[0],tokens[1:]
    #remove extension from image id
    image_id=image_id.split('.')[0]
    #convert caption list to string
    caption=" ".join(caption)
    #create list if needed
    if image_id not in mapping:
        mapping[image_id]=[]
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [6]:
len(mapping)

8091

In [7]:
def clean(mapping):
    for key,captions in mapping.items():
        for i in range(len(captions)):
            #take one caption at a time
            caption=captions[i]
            #preprocessing steps
            caption=caption.lower()#convert to lower case
            caption=caption.replace('[^A-Za-z]','')#removing special characters and digits
            caption=caption.replace(r'\s+',' ')#removing multiple spaces
            
            #add start and end tags
            caption='startseq '+" ".join([word for word in caption.split() if len(word)>1])+' endseq'
            captions[i]=caption
            
        

In [8]:
clean(mapping)
    

In [9]:
def predict_caption(model,image,tokenizer,max_length):
    #add start tag for generation
    in_text='startseq'
    #iterate over maxlength of sequence
    for i in range(max_length):
        #incode input sequence
        sequence=tokenizer.texts_to_sequences([in_text])[0]
        #pad the sequence
        sequence=pad_sequences([sequence],max_length)
        #predict next word
        yhat=model.predict([image,sequence],verbose=0)
        #convert index with high probability
        yhat=np.argmax(yhat)
        #convert index to word
        word=idx_to_word(yhat,tokenizer)
        #stop if word not found
        if word is None:
            break
        #append word as input for generation next word
        in_text +=" "+word
        #stop if we reach end tag
        if word =='endseq':
            break
    return in_text

In [10]:
all_captions=[]
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [11]:
model=tf.keras.models.load_model("best_model_70_epochs.keras")
tokenizer=Tokenizer()
tokenizer.fit_on_texts(all_captions)
max_length=35

In [12]:
#generate Captions for Image
def idx_to_word(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [16]:

image_name="97406261_5eea044056.jpg"
image_path = os.path.join('./Images/',image_name)

# load image
image = load_img(image_path, target_size=(224, 224))
# convert image pixels to numpy array
image = img_to_array(image)
# reshape data for model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# preprocess image from vgg
image = preprocess_input(image)
# extract features
img_feature = vgg_model.predict(image, verbose=0)
# predict from the trained model
predict_caption(model, img_feature, tokenizer, max_length)

InvalidArgumentError: Graph execution error:

Detected at node functional_1_1/lstm_1/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/base_events.py", line 1986, in _run_once

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/tanuj/anaconda3/lib/python3.

In [14]:
from PIL import Image
import matplotlib.pyplot as plt
def generate_caption(image_name,model,feature):
    #load Image
    #image_name="1002674143_1b742ab4b8.jpg"
    image_id=image_name.split('.')[0]
    image_path=os.path.join("Images",image_name)
    image=Image.open(image_path)
    
  
    
    #predict the caption
    y_pred=predict_caption(model,feature,tokenizer,max_length)
    
    print("--------------------------Predicted Caption----------------")
    print(y_pred)
    plt.imshow(image)

In [15]:
generate_caption(image_name,model,img_feature)

2024-12-25 04:02:57.174665: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: assertion failed: [You are passing a RNN mask that does not correspond to right-padded sequences, while using cuDNN, which is not supported. With cuDNN, RNN masks can only be used for right-padding, e.g. `[[True, True, False, False]]` would be a valid mask, but any mask that isn\'t just contiguous `True`\'s on the left and contiguous `False`\'s on the right would be invalid. You can pass `use_cudnn=False` to your RNN layer to stop using cuDNN (this may be slower).]
	 [[{{node functional_1_1/lstm_1/Assert/Assert}}]]


InvalidArgumentError: Graph execution error:

Detected at node functional_1_1/lstm_1/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/tanuj/anaconda3/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/base_events.py", line 1986, in _run_once

  File "/home/tanuj/anaconda3/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/tanuj/anaconda3/lib/python3.